In [3]:
import os
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt
from torchvision.utils import save_image
from models.gpt import GPT, GPTConfig
from jsonargparse import ArgumentParser

: 

In [4]:
import yaml
# Load YAML file
with open('configs/gpt/imagenet_64_12.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_config_args = config['model']['init_args']['config']['init_args']
gpt_config = GPTConfig(**model_config_args)
tokenizer_config_args = config['model']['init_args']['tokenizer']['init_args']
tokenizer_class = config['model']['init_args']['tokenizer']['class_path']
module_path, class_name = tokenizer_class.rsplit('.', 1)
module = __import__(module_path, fromlist=[class_name])
TokenizerClass = getattr(module, class_name)
# Instantiate the tokenizer with the config arguments
tokenizer = TokenizerClass(**tokenizer_config_args)


Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


/mnt/ndp/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/mnt/ndp/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips/vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


In [10]:
model = GPT.load_from_checkpoint("imagenet_64/gpt/epoch=299-step=375600.ckpt", config=gpt_config, tokenizer=tokenizer)
model.eval()
model.half()
model.to("cuda")

number of parameters: 216.88M


GPT(
  (transformer): OptimizedModule(
    (_orig_mod): ModuleDict(
      (wce): Embedding(1001, 768)
      (wte): Embedding(16384, 768)
      (wpe): Embedding(342, 768)
      (drop): Dropout(p=0.05, inplace=False)
      (h): ModuleList(
        (0-11): 12 x Block(
          (ln_1): LayerNorm()
          (attn): CausalSelfAttention(
            (c_attn): Linear(in_features=768, out_features=2304, bias=False)
            (c_proj): Linear(in_features=768, out_features=768, bias=False)
            (attn_dropout): Dropout(p=0.05, inplace=False)
            (resid_dropout): Dropout(p=0.05, inplace=False)
          )
          (ln_2): LayerNorm()
          (mlp): MLP(
            (c_fc): Linear(in_features=768, out_features=3072, bias=False)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=False)
            (dropout): Dropout(p=0.05, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm()
    )
  )
  (content_h

In [11]:
import torchvision
import numpy as np

labels = torch.tensor([[75]], dtype=torch.long, device=model.device)
positions = torch.tensor([[0]], dtype=torch.long, device=model.device)
z1, z2, z3, z4 = tokenizer.get_zero_tokens(1, tokenizer.embed_dim, model.device)
with torch.no_grad():
    content_tokens, positions = model.generate(labels, positions, max_new_tokens=340, temperature=1.25, cfg_scale=1.0)
    
ind4 = content_tokens[:, 1:5]
ind3 = content_tokens[:, 5:21]
ind2 = content_tokens[:, 21:85]
ind1 = content_tokens[:, 85:]

q1 = tokenizer.quantize.get_codebook_entry(ind1, shape=(1, 16, 16, tokenizer.embed_dim))
q2 = tokenizer.quantize.get_codebook_entry(ind2, shape=(1, 8, 8, tokenizer.embed_dim))
q3 = tokenizer.quantize.get_codebook_entry(ind3, shape=(1, 4, 4, tokenizer.embed_dim))
q4 = tokenizer.quantize.get_codebook_entry(ind4, shape=(1, 2, 2, tokenizer.embed_dim))

images = [tokenizer.decode(z1, z2, z3, q4), tokenizer.decode(z1, z2, q3, q4), tokenizer.decode(z1, q2, q3, q4), tokenizer.decode(q1, q2, q3, q4)]
images = torch.cat((images[0], images[1], images[2], images[3])).detach().cpu()
images = torch.clamp(images, -1., 1.)

grid = torchvision.utils.make_grid(images, nrow=4)
grid = (grid+1.0)/2.0 # -1,1 -> 0,1; c,h,w
grid = grid.transpose(0,1).transpose(1,2).squeeze(-1)
grid = grid.numpy()
grid = (grid*255).astype(np.uint8)
from PIL import Image
import matplotlib.pyplot as plt
# Convert numpy array to PIL Image
img = Image.fromarray(grid)
# Display the image in the notebook
plt.figure(figsize=(12, 3))
plt.imshow(img)
plt.axis('off')
plt.show()


  0%|          | 0/340 [00:00<?, ?it/s]

content_idx_cond torch.Size([1, 1]) torch.int64
position_idx_cond torch.Size([1, 1]) torch.int64


RuntimeError: mat1 and mat2 shapes cannot be multiplied (12x64 and 768x768)